In [ ]:
"""
You need to run this cell for the code in following cells to work.
"""

# Enable module reloading
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import numpy as np

from week_4.backstage.load_data import load_data

%load_ext tensorboard
%tensorboard --logdir logs --bind_all

# Week 5

__Goals for this week__

We will learn about _hyperparameter tuning,_ an important part of each machine learning project.

__Feedback__

This lab is a work in progress. If you notice a mistake, notify us or you can even make a pull request. Also please fill the [questionnaire](https://forms.gle/r27nBAvnMC7jbjJ58) after you finish this lab to give us feedback.

## Hyperparameter tuning

In previous labs we introduced various quantities that control how does the model train or how does the model look like, e.g. _learning rate,_ _batch size,_ _hidden layer size_ and others. These quantities are called _hyperparameters_. In contrast to regular parameters, they are set before the training takes place and they are not modified during the gradient descent training. The values we choose for the hyperparameters can significantly change the performance of the model.


__Exercise 5.1:__ First, check the code in `mlp.py`. You should find it familiar, it is based on the code from the last week's lab. We abstracted all the options we might want to play with into a `hparams` dictionary of the `train` function.

1. Try to manually set the hyperparameter values in the code below. You should be able to get 95% accuracy in 50 epochs with proper hyperparameters.
2. You can add your code that adds customizable number of layer from previous lab. Number of layers should also be a hyperparameter.
3. Check your results in [TensorBoard](http://localhost:6006). See the __HPARAMS__ page, it has great visualizations of your runs.

In [ ]:
from week_5.mlp import train

data = load_data('iris.csv', num_classes=3)

val_acc, loss = train(data.x, data.y,
    dim_output=3,  # This is actually not a hyperparameter, it just describes our data.
    dim_hidden=30,
    learning_rate=0.001,
    batch_size=64,
    activation='sigmoid',
    output_activation='linear',
    loss_function='mse',
    epoch=50)

print(f'Final loss: {loss:.3f} Best validation accuracy: {val_acc:.3f}')

_Hyperparameter tuning,_ a process of searching for better hyperparameter values, should be done during each machine learning project. We expect you to tune your hyperparameters during your course projects as well. The goal of hyperparameter tuning is to find a set of hyperparameters that optimizes some evaluation metric, e.g. for classification we want to find hyperparametes that has the best accuracy of the model. We do this by searching through space of all possibel values either via manual tuning or via automatic tuning.

### Manual tuning

Manual tuning is a name for a trial-and-error tuning that is done by hand. You essentialy look at how did the model train and try to guess which hyperparameter needs to change. Then you train a model with new hyperparameters and see how did it affect the results. You repeat this until you are content with the performance of your model.

This process is of course quite unsystematic and ineffective. Even for professionals it is hard to guess which hyperparameters are problematic. With this technique you also often explore only a small subspace of all possible values and perhaps you might miss global optimum entirely.

### Random search tuning

Random search is a more systematic approach. You set a interval of possible values for each hyperparameter. Then you randomly sample from these intervals and train the model with what you sampled. You repeat this process until you are content with the performance, or until you have computational resources.

This approach demands more compute than the manual tuning, but it can be fully automatized. Getting better performance is a function of time in this case, you are bound to find better and better models as the time goes on. How you set the intervals for each hyperparameter and how you sample from this interval is very important. Setting these wrong can make the process inefficient. We discuss these questions in the next section.
 
### Hyperparameter properties

Hyperparameters can be divided by their type:

- _Integer hyperparameters,_ e.g. hidden layer size, number of hidden layers
- _Real number hyperparameters,_ e.g. learning rate
- _Categorical hyperparameter,_ e.g. activation function

For number hyperparameters (both integer and real) we sample from a pre-defined range during random search. E.g. for hidden layer size we might define the minimum value as 10 and the maximum value as 1000. We then pick the value from within this range. There are two basic ways of sampling number from within this range:

- _Linear,_ when we simply pick random value from within this range using uniform distribution.
- _Exponential,_ when we define the range via exponents as $\langle 10^1, 10^3\rangle$. Instead of sampling from 10 to 1000, we sample from 1 to 3 interval. This skews the distribution towards the smaller numbers, e.g. in this case half of the values will fall into $\langle 10, 100 \rangle$ interval, while the other half will fall into $\langle 100, 1000 \rangle$ interval, even though the second interval is in fact 10 times bigger.

Below we list some hyperparameters you already encountered with recommended starting ranges. They are sorted by what we consider to be an order of how important each parameter is for tuning. This order is quite subjective and other practiotioneers might have different opinions.

__Learning rate__ - real - exponential - $\langle 10^{-2}, 10^{-4} \rangle$.  
Learning rate is the most important hyperparameter that should always be tuned. Setting it too low will halt the training as it can get stuck in plateaus or it can pointlessly make the training longer. Setting it too high might cause divergence (see Week 2 lab) that can lead to numeric overflow exception.

__Batch size__ - integer - exponential - $\langle 2^3, 2^6 \rangle$.  
Using larger batch size provides faster training as we can use parallelism abilities of modern HW. However the model performance can decrease. Some more complex tasks (e.g. game playing bot) are trained with extremely big batch size in the order of millions.

__Hidden layer size__ - integer - exponential - $\langle 2^5, 2^8 \rangle$.  
Setting the hidden layer too small can low model capacity. Capacity is the ability of ML models to model the data. E.g. linear regression has very small capacity, because it can only model linear relations. Setting the hidden layer too big can cause overfitting. We will talk about overfitting in following labs.

__Number of layers__ - integer - linear - $\langle 1, 5 \rangle$.  
Compared to previous hyperparameters, number of layers if often architecture specific. For MLP model we learned about so far we usually work with relatively small number of layers. More layers are often used in computer vision convolutional neural networks.

__Activation function__ - categorical - { relu, sigmoid, ... }.  
Activation function can be experimented with, but is usually not so important. ReLU is usually a good starting point. For really small models you can use sigmoid instead.

__Loss function__ - categorical - { cross-entropy + softmax activation, MSE + linear activation, ... }  
There are some loss functions that are usually used for some tasks, e.g. you should use cross-entropy with softmax for classification or MSE for regression. 

There are multiple aspects we need to take into consideration when we choose the hyperparameters, e.g. batch size, layer size and number of layers also influence how big is the model memory-wise. Setting these parameters too high can make the it too big for training on available hardware. We are usually limited by the size of RAM in our HW accelerators (e.g. GPU cards).

__Exercise 5.2:__ Setup a random search hyperparameter optimization in the code below. Follow the instructions above to see the value range and sampling technique for each hyperparameter. Observe how you get better and better results over time.

In [ ]:
# Quick random number generator to get you started

batch_size = lambda: np.random.randint(1, 128)
print(batch_size())
print(batch_size())

In [ ]:
max_acc = 0
run_id = 0

for _ in range(100):
    acc, _ = train(data.x, data.y,
        dim_output=3,  # This is actually not a hyperparameter, it just describes our data.
        dim_hidden=30,
        learning_rate=0.001,
        batch_size=batch_size(),
        activation='sigmoid',
        output_activation='linear',
        loss_function='mse',
        epoch=50)
    
    if acc > max_acc:
        max_acc = acc
        print(f'New best in epoch {run_id} score: {acc:.3f}, Hyperparameters: {...}')
        
    run_id += 1

### Tuning in practice

1. You usually start with manual tuning during the development, when you just want to quickly see whether the model works and is able to learn. As soon as you have your model ready and you want to properly train and evaluate it, you should switch to random tuning.

2. Check the hyperparameter values people use in recent (2014 or later) and related (same dataset or task) projects. They should server as a fine starting point.

3. You can gradually change the search intervals. E.g. if you find out that a certain subspace has good results, you can focus on this subspace. Similarly, you can expand the range of some parameter, if the best results are achieved with its marginal values. E.g. with batch sizes 4, 8 and 16 you always have the best results with 16. Then it makes sense to expand the batch size range to 32 and perhaps 64 as well.

## Further reading

- Alessio Gozzoli has a nice [blog](https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/) about various hyperparameter tuning techniques and their practical aspects.
- Andrew Ng discusses hyperparameter tuning in his Coursera's Deep Learning course in three  consecutive videos (21 minutes together): [1](https://youtu.be/AXDByU3D1hA), [2](https://youtu.be/cSoK_6Rkbfg), [3](https://youtu.be/wKkcBPp3F1Y).
